In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install rawpy 
!pip install scipy==1.2.0

     |████████████████████████████████| 1.6MB 3.5MB/s 
     |████████████████████████████████| 26.6MB 113kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
# uniform content loss + adaptive threshold + per_class_input + recursive G
# improvement upon cqf37
%tensorflow_version 1.x
from __future__ import division

import os, time, scipy.io
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import itertools 
import rawpy
import glob

input_dir1 = '/gdrive/My Drive/Sony11/Sony/short1/'
input_dir2 = '/gdrive/My Drive/Sony11/Sony/short2/'
gt_dir1 = '/gdrive/My Drive/Sony11/Sony/long1/'
gt_dir2 = '/gdrive/My Drive/Sony11/Sony/long2/'
checkpoint_dir = '/gdrive/My Drive/Sony11/Sony/result_Sony/'
result_dir = '/gdrive/My Drive/Sony11/Sony/result_Sony/'

# get train IDs
train_fns1 = glob.glob(gt_dir1 + '0*.ARW')
train_ids1 = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns1]
#train_ids1.sort()
print('train_ids1:',train_ids1)
train_fns2 = glob.glob(gt_dir2 + '0*.ARW')
train_ids2 = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns2]
#train_ids2.sort()
print('train_ids2:',train_ids2)

ps = 512          # patch size for training
save_freq = 20


DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]


def lrelu(x):
    return tf.maximum(x * 0.2, x)


def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])
   
    return deconv_output


def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    #print("up6::::::::::",up6.shape)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 2)
    #print('out image',out.shape)
    return out


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out


sess = tf.Session()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
out_image = network(in_image)

G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))
print(G_loss)

t_vars = tf.trainable_variables()
lr = tf.placeholder(tf.float32)
G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

# Raw data takes long time to load. Keep them in memory after loaded.

length_1=len(train_ids1)
length_2=len(train_ids2)
length=length_1+length_2
gt_images1 = [None] * length_1
gt_images2 = [None] * length_1
input_images1 = {}
idss=train_ids1 + train_ids2
#print('all ids',idss)
'''input_images1['300'] = [None] * length_1
input_images1['250'] = [None] * length_1
input_images1['100'] = [None] * length_1

input_images2 = {}
input_images2['300'] = [None] * length_2
input_images2['250'] = [None] * length_2
input_images2['100'] = [None] * length_2'''


print('length::::',length)
print('length1::::',length_1)
print('length2::::',length_2)
#print('input_images1::::',input_images1)
#print('gt_images1::::',gt_images1)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))

learning_rate = 1e-4
for epoch in range(lastepoch, 4001):
    if os.path.isdir(result_dir + '%04d' % epoch):
        continue
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5
    for shimgs in range(0,2):
        if(shimgs==0):
         train_ids=train_ids1
         input_dir=input_dir1
         gt_dir=gt_dir1
         gt_images = gt_images1# [None] * length_1
         input_images1['300'] = [None] * 1
         input_images1['250'] = [None] * 1
         input_images1['100'] = [None] * 1
         input_images=input_images1
        else:
          train_ids=train_ids2
          input_dir=input_dir2 
          gt_dir=gt_dir2
          input_images1['300'] = [None] * 1
          input_images1['250'] = [None] * 1
          input_images1['100'] = [None] * 1
          input_images=input_images1
          gt_images = gt_images2#[None] * length_2
          
        ff=0
        #print('length_train_ids',len(train_ids))
        #print('train_ids',train_ids)  
        #print('input_images',input_images)        
        for ind in range(0,len(train_ids)):
           # get the path from image id
            #print('index:',ind)
            ff+=1
            #print('ff: ',ff)
            train_id = train_ids[ind]
            print('image id %d'% train_id)
            in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
            #print('infiles: ',in_files)
            in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
            #print('inpath: ',in_path)
            in_fn = os.path.basename(in_path)

            gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
            gt_path = gt_files[0]
            #print('gtpath: ',gt_path)
            gt_fn = os.path.basename(gt_path)
            in_exposure = float(in_fn[9:-5])
            gt_exposure = float(gt_fn[9:-5])
            ratio = min(gt_exposure / in_exposure, 300)
            #print('ratio',ratio)

            st = time.time()
            cnt += 1
           
            
            #if input_images[str(ratio)[0:3]][ind] is None:
            raw = rawpy.imread(in_path)
            input_images[str(ratio)[0:3]][0] = np.expand_dims(pack_raw(raw), axis=0) * ratio

            if(ff==81 or ff==80 or ff==79):
                print('inside if :',ff, ind)
                gt_raw = rawpy.imread(gt_path)
                im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                gt_images[78] = np.expand_dims(np.float32(im / 65535.0), axis=0)
            else:    
              if gt_images[ind] is None:
                  gt_raw = rawpy.imread(gt_path)
                  im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                  gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)

            # crop
            H = input_images[str(ratio)[0:3]][0].shape[1]
            W = input_images[str(ratio)[0:3]][0].shape[2]

            xx = np.random.randint(0, W - ps)
            yy = np.random.randint(0, H - ps)
            input_patch = input_images[str(ratio)[0:3]][0][:, yy:yy + ps, xx:xx + ps, :]

            if(ff==81 or ff==80 or ff==79):
              gt_patch = gt_images[78][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]
            else:
              gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]
            #print('input_patch:: ',input_patch.shape)
            #print('gt_patch:: ',gt_patch.shape)
            #print(input_patch)

            if np.random.randint(2, size=1)[0] == 1:  # random flip
                input_patch = np.flip(input_patch, axis=1)
                gt_patch = np.flip(gt_patch, axis=1)
            if np.random.randint(2, size=1)[0] == 1:
                input_patch = np.flip(input_patch, axis=2)
                gt_patch = np.flip(gt_patch, axis=2)
            if np.random.randint(2, size=1)[0] == 1:  # random transpose
                input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

            input_patch = np.minimum(input_patch, 1.0)
            #print('inp2',input_patch.shape)
            #print('gt2',gt_patch.shape)
            #print(input_patch)

            _, G_current, output = sess.run([G_opt, G_loss, out_image],
                                            feed_dict={in_image: input_patch, gt_image: gt_patch, lr: learning_rate})
            #print("output_before::",output.shape)
            output = np.minimum(np.maximum(output, 0), 1)
            g_loss[ind] = G_current
            #print("output shape::",output.shape)

            print("%d %d Loss=%.3f Time=%.3f" % (epoch, cnt, np.mean(g_loss[np.where(g_loss)]), time.time() - st))

            if epoch % save_freq == 0:
                if not os.path.isdir(result_dir + '%04d' % epoch):
                    os.makedirs(result_dir + '%04d' % epoch)

                temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :, :]), axis=1)
                scipy.misc.toimage(temp * 255, high=255, low=0, cmin=0, cmax=255).save(
                    result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))
        
        
        #if(flag==1):
           #input_images1=input_images
           #print('input_images1:::',input_images1)
        #elif(flag==2):
           #input_images2=input_images
           #print('input_images2:::',input_images2)


       
    saver.save(sess, checkpoint_dir + 'model.ckpt')

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

train_ids1: [90, 91, 92, 72, 73, 75, 76, 78, 81, 83, 84, 85, 86, 88, 94, 95, 96, 97, 98, 99, 100, 102, 104, 108, 109, 110, 112, 113, 114, 117, 1, 2, 4, 9, 10, 12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 26, 27, 28, 29, 31, 33, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 70, 71]
train_ids2: [118, 119, 121, 122, 123, 124, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 141, 142, 143, 144, 145, 146, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 164, 165, 166, 168, 169, 171, 173, 174, 175, 179, 180, 181, 182, 183, 184,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:208: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead


3941 1 Loss=0.043 Time=14.011
image id 91
3941 2 Loss=0.034 Time=9.223
image id 92
3941 3 Loss=0.032 Time=3.606
image id 72
3941 4 Loss=0.028 Time=3.012
image id 73
3941 5 Loss=0.025 Time=3.223
image id 75
3941 6 Loss=0.024 Time=3.145
image id 76
3941 7 Loss=0.024 Time=3.476
image id 78
3941 8 Loss=0.027 Time=3.006
image id 81
3941 9 Loss=0.026 Time=2.834
image id 83
3941 10 Loss=0.026 Time=3.398
image id 84
3941 11 Loss=0.026 Time=3.482
image id 85
3941 12 Loss=0.026 Time=3.335
image id 86
3941 13 Loss=0.025 Time=4.063
image id 88
3941 14 Loss=0.024 Time=2.552
image id 94
3941 15 Loss=0.025 Time=3.281
image id 95
3941 16 Loss=0.025 Time=3.632
image id 96
3941 17 Loss=0.025 Time=4.068
image id 97
3941 18 Loss=0.026 Time=3.523
image id 98
3941 19 Loss=0.026 Time=4.616
image id 99
3941 20 Loss=0.026 Time=3.655
image id 100
3941 21 Loss=0.026 Time=2.740
image id 102
3941 22 Loss=0.025 Time=3.014
image id 104
3941 23 Loss=0.025 Time=3.880
image id 108
3941 24 Loss=0.025 Time=3.240
image id

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:208: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead


3941 31 Loss=0.023 Time=3.824
image id 2
3941 32 Loss=0.023 Time=3.459
image id 4
3941 33 Loss=0.022 Time=3.293
image id 9
3941 34 Loss=0.022 Time=2.942
image id 10
3941 35 Loss=0.022 Time=2.949
image id 12
3941 36 Loss=0.022 Time=3.221
image id 13
3941 37 Loss=0.021 Time=3.272
image id 14
3941 38 Loss=0.021 Time=3.668
image id 15
3941 39 Loss=0.021 Time=3.520
image id 17
3941 40 Loss=0.021 Time=3.688
image id 18
3941 41 Loss=0.021 Time=4.927
image id 19
3941 42 Loss=0.021 Time=3.466
image id 21
3941 43 Loss=0.020 Time=2.632
image id 23
3941 44 Loss=0.020 Time=2.607
image id 24
3941 45 Loss=0.020 Time=2.957
image id 26
3941 46 Loss=0.020 Time=4.321
image id 27
3941 47 Loss=0.020 Time=3.781
image id 28
3941 48 Loss=0.020 Time=2.455
image id 29
3941 49 Loss=0.020 Time=3.180
image id 31
3941 50 Loss=0.020 Time=2.804
image id 33
3941 51 Loss=0.020 Time=2.658
image id 36
3941 52 Loss=0.020 Time=3.845
image id 37
3941 53 Loss=0.020 Time=2.760
image id 38
3941 54 Loss=0.020 Time=3.827
image i

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:208: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead


Streaming output truncated to the last 5000 lines.
3944 117 Loss=0.017 Time=0.981
image id 164
3944 118 Loss=0.017 Time=1.039
image id 165
3944 119 Loss=0.017 Time=1.032
image id 166
3944 120 Loss=0.017 Time=1.045
image id 168
3944 121 Loss=0.017 Time=1.017
image id 169
3944 122 Loss=0.017 Time=1.007
image id 171
3944 123 Loss=0.017 Time=1.042
image id 173
3944 124 Loss=0.017 Time=1.022
image id 174
3944 125 Loss=0.017 Time=1.042
image id 175
3944 126 Loss=0.017 Time=1.023
image id 179
3944 127 Loss=0.017 Time=1.018
image id 180
3944 128 Loss=0.017 Time=1.017
image id 181
3944 129 Loss=0.017 Time=0.918
image id 182
3944 130 Loss=0.017 Time=1.037
image id 183
3944 131 Loss=0.017 Time=0.982
image id 184
3944 132 Loss=0.017 Time=1.174
image id 186
3944 133 Loss=0.018 Time=1.807
image id 189
3944 134 Loss=0.018 Time=1.093
image id 190
3944 135 Loss=0.018 Time=0.999
image id 194
3944 136 Loss=0.018 Time=1.047
image id 195
3944 137 Loss=0.018 Time=1.009
image id 196
3944 138 Loss=0.018 Time=

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:285: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


Streaming output truncated to the last 5000 lines.
3985 116 Loss=0.016 Time=1.029
image id 161
3985 117 Loss=0.016 Time=1.008
image id 164
3985 118 Loss=0.016 Time=1.021
image id 165
3985 119 Loss=0.016 Time=1.029
image id 166
3985 120 Loss=0.016 Time=1.025
image id 168
3985 121 Loss=0.016 Time=1.037
image id 169
3985 122 Loss=0.016 Time=1.006
image id 171
3985 123 Loss=0.016 Time=1.019
image id 173
3985 124 Loss=0.016 Time=1.014
image id 174
3985 125 Loss=0.016 Time=1.047
image id 175
3985 126 Loss=0.016 Time=1.007
image id 179
3985 127 Loss=0.016 Time=1.025
image id 180
3985 128 Loss=0.016 Time=1.027
image id 181
3985 129 Loss=0.016 Time=1.008
image id 182
3985 130 Loss=0.016 Time=1.062
image id 183
3985 131 Loss=0.016 Time=0.988
image id 184
3985 132 Loss=0.017 Time=1.013
image id 186
3985 133 Loss=0.017 Time=1.019
image id 189
3985 134 Loss=0.017 Time=1.038
image id 190
3985 135 Loss=0.017 Time=1.009
image id 194
3985 136 Loss=0.017 Time=1.128
image id 195
3985 137 Loss=0.018 Time=